In [6]:
from torch.utils.data import random_split, DataLoader
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms


In [7]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


full_dataset = datasets.ImageFolder(root='animal_data', transform=train_transform)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

class_names = full_dataset.classes



In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  # Reset gradients
            
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights
            
            running_loss += loss.item()
            
            if batch_idx % 20 == 19:
                print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
        
        epoch_train_loss = running_loss / len(train_loader)
        train_losses.append(epoch_train_loss)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                
                # Calculate accuracy
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        epoch_val_loss = val_loss / len(val_loader)
        epoch_val_accuracy = 100 * correct / total
        
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_accuracy)
        
        print(f'Epoch [{epoch+1}/{num_epochs}] Complete:')
        print(f'  Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_accuracy:.2f}%')
        print('-' * 50)
    
    return train_losses, val_losses, val_accuracies


In [18]:
import torchvision.models as models
import torch.nn as nn
from sklearn.metrics import f1_score, recall_score, precision_score
model = models.resnet18(weights='IMAGENET1K_V1')

for param in model.parameters():
    param.requires_grad = False

for param in list(model.parameters())[-5:]:
    param.requires_grad = True

num_features = model.fc.in_features 
model.fc = nn.Linear(num_features, 15)
model = model.to(device)

print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  

train_losses, val_losses, val_accuracies = train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=5
)

def eval_metrics(model, val_loader):
    model.eval()
    pred = []
    t_val = []

    with torch.no_grad():
        for inputs, val in val_loader:
            inputs, val = inputs.to(device), val.to(device)
            output = model(inputs)

            prediction = output.argmax(dim=1)

            pred.append(prediction.cpu().numpy())
            t_val.append(val.cpu().numpy())

    predictions_array = np.concatenate(pred)
    t_val_array = np.concatenate(t_val)
    
    precision = precision_score(t_val_array, predictions_array, average='macro')*100
    recall = recall_score(t_val_array, predictions_array, average='macro')*100
    f1 = f1_score(t_val_array, predictions_array, average='macro')*100
    accuracy = np.mean(predictions_array==t_val_array)*100
    print(f"Accuracy  : {accuracy}\n"
          f"Precision : {precision}\n"
          f"Recall    : {recall}\n"
          f"F1        : {f1}")

eval_metrics(model, val_loader)


Trainable parameters: 2368015
Total parameters: 11184207
Epoch [1/5], Batch [20/49], Loss: 0.2914
Epoch [1/5], Batch [40/49], Loss: 0.2434
Epoch [1/5] Complete:
  Train Loss: 0.5499, Val Loss: 0.1445, Val Acc: 96.40%
--------------------------------------------------
Epoch [2/5], Batch [20/49], Loss: 0.0642
Epoch [2/5], Batch [40/49], Loss: 0.0596
Epoch [2/5] Complete:
  Train Loss: 0.0835, Val Loss: 0.0877, Val Acc: 97.94%
--------------------------------------------------
Epoch [3/5], Batch [20/49], Loss: 0.0209
Epoch [3/5], Batch [40/49], Loss: 0.0320
Epoch [3/5] Complete:
  Train Loss: 0.0380, Val Loss: 0.0939, Val Acc: 96.92%
--------------------------------------------------
Epoch [4/5], Batch [20/49], Loss: 0.0154
Epoch [4/5], Batch [40/49], Loss: 0.0186
Epoch [4/5] Complete:
  Train Loss: 0.0199, Val Loss: 0.0790, Val Acc: 98.20%
--------------------------------------------------
Epoch [5/5], Batch [20/49], Loss: 0.0236
Epoch [5/5], Batch [40/49], Loss: 0.0058
Epoch [5/5] Compl